# DS Bowl 2019 : Explore How Young Children Learn

Exploration based on [Kaggle DS Bowl 2019](https://www.kaggle.com/c/data-science-bowl-2019) competition dataset.

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Load Data

Data specification:

__train.csv:__
_Gameplay events._

* `event_id` - Randomly generated unique identifier for the event type. Maps to `event_id` column in specs table.
* `game_session` - Randomly generated unique identifier grouping events within a single game or video play session.
* `timestamp` - Client-generated datetime
* `event_data` - Semi-structured JSON formatted string containing the events parameters. Default fields are: `event_count`, `event_code`, and `game_time`; otherwise fields are determined by the event type.
* `installation_id` - Randomly generated unique identifier grouping game sessions within a single installed application instance.
* `event_count` - Incremental counter of events within a game session (offset at 1). Extracted from `event_data`.
* `event_code` - Identifier of the event `class`. Unique per game, but may be duplicated across games. E.g. event code `2000` always identifies the `Start Game` event for all games. Extracted from `event_data`.
* `game_time` - Time in milliseconds since the start of the game session. Extracted from `event_data`.
* `title` - Title of the game or video.
* `type` - Media type of the game or video. Possible values are: `Game`, `Assessment`, `Activity`, `Clip`.
* `world` - The section of the application the game or video belongs to. Helpful to identify the educational curriculum goals of the media. Possible values are: `NONE` (at the app's start screen), `TREETOPCITY` (Length/Height), `MAGMAPEAK` (Capacity/Displacement), `CRYSTALCAVES` (Weight).

__specs.csv:__
_Specification of the various event types._

* `event_id` - Global unique identifier for the event type. Joins to `event_id` column in events table.
* `info` - Description of the event.
* `args` - JSON formatted string of event arguments. Each argument contains:
    * `name` - Argument name.
    * `type` - Type of the argument (string, int, number, object, array).
    * `info` - Description of the argument.

In [4]:
logs = pd.read_csv('./data/train.csv')
specs = pd.read_csv('./data/specs.csv')

In [5]:
logs.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [6]:
specs.head()

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


## Questions

`1` a. Find top-5 most active users (`installation_id`) in August 2019 (`timestamp`) by the number of events

In [12]:
# split timestamp into several columns
logs_split = logs.copy()
logs_split[['year', 'month', 'daytime']] = logs.timestamp.str.split('-', expand=True)

In [15]:
# filter the dataset by timestamp and find the number of events for each installation_id
active_users_events = logs_split[(logs_split.year == '2019') & (logs_split.month == '08')]\
.installation_id.value_counts()

active_users_events[:5]

c9ea32e9    29324
96979008    25404
f1c21eda    25217
1f71eb61    23225
d25338cb    22671
Name: installation_id, dtype: int64

`1` b. Find top-5 most active users (`installation_id`) in August 2019 (`timestamp`) by the number of sessions

In [27]:
# group by user (installation_id) and game session
# reset the index
active_user_sessions = logs_split[(logs_split.year == '2019') & (logs_split.month == '08')]\
.groupby(by=['installation_id','game_session']).count()[['event_id']].reset_index()

In [29]:
# add the counter column
active_user_sessions['counts'] = 1

In [36]:
# group by the installatio_id and aggregate over the counter column
active_user_sessions.groupby(by=['installation_id']).sum()[['counts']].sort_values(by=['counts'], ascending=False)\
.head(5) # view the top-5 results

,counts
installation_id,
08987c08,889
f1c21eda,520
43249190,490
f7ffd521,451
2ffb27f9,389


* The top-1 user in terms of the number of events is not the top user in terms of sessions.
* The top-2 user in terms of events if the top-1 user in terms of sessions also.

`2` Which assessment is the most complicated?

Assessment attempts are captured in event_code 4100 for all assessments except for Bird Measurer and 4110 for Bird Measurer (use `title != 'Bird Measurer (Assessment)` ). If the attempt was correct, it contains "correct":true.

In [50]:
# example of the successful attempt
logs[(logs.event_code == 4100 ) | (logs.event_code == 4110) ].event_data.values[0] # Bird Measurer check to be added

'{"correct":true,"stumps":[1,2,4],"event_count":44,"game_time":31011,"event_code":4100}'

In [70]:
# get the df containing only attempts
attempts = logs[((logs.event_code == 4100 ) & (logs.title != 'Bird Measurer (Assessment)')) | ((logs.event_code == 4110) & (logs.title == 'Bird Measurer (Assessment)'))]

def get_success(row):
    '''
    Helper function to get the result of the attempt
    '''
    json = row.event_data
    try:
        res = pd.read_json(json)
    except:
        return np.nan
    
    if len(res.correct.values) == 0:
        return np.nan
        
    return 1 if res.correct.values[0] else 0

# get the attempt results
attempts['success'] = attempts.apply(lambda row: get_success(row), axis = 1)

/Users/aleksandraastakhova/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [73]:
# remove the empty rows not belonging to the assessments
attempts = attempts.dropna()

In [76]:
# group the attempts records by time
# find the average passing rate
attempts.groupby(by=['title']).mean().sort_values(by=['success'], ascending=True)

,event_count,event_code,game_time,success
title,,,,
Bird Measurer (Assessment),53.501344,4110.0,99329.588815,0.166697
Chest Sorter (Assessment),60.265146,4100.0,89987.640060,0.168842
Cauldron Filler (Assessment),32.047804,4100.0,72240.454472,0.486610
Mushroom Sorter (Assessment),45.392235,4100.0,53222.946797,0.505352
Air Show,104.829240,4100.0,222104.519803,0.658009
Cart Balancer (Assessment),29.623434,4100.0,34788.696143,0.811840


__Bird Measurer assessment seems to be the most complicated. The average passing rate is just 16%.__

`3` Create a pivot table for number of event type vs month (hint: use `pivot_table` function)

Example:

| Month/Type        | 07           | 08  |
| ------------- |:-------------:| -----:|
| Activity     | 100| 100 |
| Assessment      | 100      |   100 |
| Clip | 100      |    100 |

In [89]:
# add a dummy column for a counter
logs_split['counter'] = 1

In [91]:
# create a pivot table to find the number of events of each type per month
pd.pivot_table(logs_split, index=['type'], columns=['month'], values = 'counter', aggfunc=np.sum)

month,07,08,09,10
type,,,,
Activity,285994,1481486,1794533,874715
Assessment,66416,320733,364346,173850
Clip,12082,59690,75590,36314
Game,347502,1988204,2352339,1107248


`4` Binarize `game_time` column into following bins:
* `early` - < 30000
* `mid` - >= 30000  and < 70000
* `late` - >= 70000

What is the difference between `cut` and `qcut` functions?

In [99]:
# just view the statistics
logs_split.game_time.to_frame().describe()

,game_time
count,1.134104e+07
mean,1.976236e+05
std,1.803968e+06
min,0.000000e+00
25%,3.061800e+04
50%,7.276100e+04
75%,1.574460e+05
max,3.069102e+08


In [105]:
# define the labels
cut_labels = ['early', 'mid', 'late']
# define the bin borders
cut_bins = [-1, 30000, 70000, 306910249]
# create binarized column
logs_split['cut_game_time'] = pd.cut(logs_split['game_time'], bins=cut_bins, labels=cut_labels)

In [107]:
logs_split.head(1)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,year,month,daytime,counter,cut_game_time
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,2019,09,06T17:53:46.937Z,1,early
